In [8]:
import sqlite3, time
from pandas.io import sql
import yfinance as yf
import ta
import requests
import pandas as pd
import numpy as np
import datetime,time,re
import mplfinance as mpf
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression

In [9]:
dict_from_ETF_list = pd.read_excel("ETF_list.xlsx")
ETF_list = dict_from_ETF_list['Number']

today = datetime.date.today()
end = today - datetime.timedelta(days=1)
start = end - datetime.timedelta(days=365*3.5)

In [10]:
conn = sqlite3.connect('ETF_list.db')
cursor = conn.cursor()

In [11]:
conn.execute('''CREATE TABLE if not exists ETF_data
       (Date date ,
       Number TEXT ,
       Open FLOAT,
       High FLOAT,
       Low FLOAT,
       Close FLOAT,
       Volume TEXT,
       primary key (Date, Number));''')
conn.commit()

In [13]:
df = yf.download("F0HKG05WWU:FO", start = str(start), end = str(end))

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- F0HKG05WWU:FO: No data found, symbol may be delisted


In [12]:
#ETF_list = ['0050']
for i in ETF_list:
    print(i)
    df = yf.download(f"{i}.TW", start = str(start), end = str(end))
    if df.empty:
            print('no data today')
            continue
    df['Number'] = f'{i}'
    df['Date'] = df.index.strftime("%Y-%m-%d")
    pick_df = df[['Date', 'Number','Open', 'High', 'Low', 'Close', 'Volume']]

    #print(pick_df.index)
    #print(pick_df)
    #print('股票：',i, pick_df)
    pick_df['Volume'] = pick_df['Volume'].map('{}'.format)

    #print(tuple(pick_df.iloc[0]))
    #print(len(pick_df))

    for i in range(len(pick_df)):
        #pick_df[i][6] = pick_df[i][6].astype(str)
        #print(type(pick_df.iloc[i][6]))
        #pick_df.iloc[i][6] = str(pick_df.iloc[i][6])
        #a = "{}".format(pick_df.iloc[i][6])
        #print(a)
        #print(type(a))
        #pick_df.iloc[i][6] = 10
        #pick_df[i]['Volume'] = 
        #print(pick_df.iloc[i][6])
        #print(type(pick_df.iloc[i][6]))
        cursor.execute('insert into ETF_data(Date, Number, Open, High, Low, Close, Volume) VALUES (?, ?, ?, ?, ?, ?, ?)', tuple(pick_df.iloc[i]))
        conn.commit()
        #(pick_df[0][0], pick_df[0][1], ,,,)

0050
[*********************100%***********************]  1 of 1 completed


C:\Users\willi\AppData\Local\Temp/ipykernel_24820/184870118.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pick_df['Volume'] = pick_df['Volume'].map('{}'.format)


IntegrityError: UNIQUE constraint failed: ETF_data.Date, ETF_data.Number

## sql = "select * from ETF_data where Number = '0050'"
cursor.execute(sql)
rows = cursor.fetchall()
print(rows)

In [7]:
#刪除資料表
sql = "DROP TABLE ETF_data"
cursor.execute(sql)

OperationalError: near "|": syntax error